In [1]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl
import glob

import datetime
today = date.today()
# today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

time = "7pm_"

In [17]:
## DKNG Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

df = pd.read_csv(r'C:/Users/vchang/Downloads/DKEntries.csv', header=7, usecols=[13,15,16,17,18,19,20,21])
df.to_csv("DKSalaries_"+ contest + time + str(today) +".csv", index=False)

shutil.move(downloads + "/"+ "DKSalaries_"+ contest + time + str(today) +".csv",
           contest_data_loc + "/"+ "DKSalaries_"+ contest + time + str(today) +".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
# dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + time + str(today) +".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + time + str(today) +".csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + time + str(today) +".xlsx", index=False)

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
os.remove("DKSalaries_"+ contest + time + str(today) +".csv")


## Uploads
## Uploads
os.chdir(r"C:\Users\vchang\Downloads")

draftkings_upload = r"C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\DraftKings\Template"

dk_upload = glob.glob('DKEntries.csv')[0]
os.rename(dk_upload, "DK_Upload_"+ contest + time + str(today) +".csv")

shutil.move(downloads + "/"+ "DK_Upload_"+ contest + time + str(today) +".csv",
           draftkings_upload + "/"+ "DK_Upload_"+ contest + time + str(today) +".csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/vchang/Downloads/DKEntries.csv'

In [16]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/DraftKings'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+ time + str(today)+".xlsx", index=False)
os.remove("DK NBA Projections.csv")

In [4]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\DraftKings\ETR_Daily_' + time + str(today)  + ".xlsx")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + time + str(today)  + ".xlsx")


dkng['ID'] = dkng['ID'].astype(str)
dkng['Name + ID'] = dkng['Name'] + ' (' + dkng['ID'] + ')'

df = pd.merge(etr, 
              dkng[['Name','Roster Position', 'Name + ID', 'Salary']], 
              left_on='Player', 
              right_on='Name')



## Unnests Roster Positions into each Row (explode in Python) ##

# Update Salary to INT 
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

# Update Roster Position to Str
df['Roster Position'] = df['Roster Position'].astype(str)

# Delimit the text by /
df['Roster Position'] = df['Roster Position'].str.split('/')

df = df.explode('Roster Position')

df.head()

df.to_csv("DKNG_final_PlayerPool_ETR_"+ time + str(today) +".csv", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+ time + str(today) +".xlsx")

In [5]:
## Optimizer

df = pd.read_csv("DKNG_final_PlayerPool_ETR_"+ time + str(today) +".csv", index_col=['Name + ID', 'Roster Position'], skipinitialspace=True)

legal_assignments = df.index # tuples of (name, pos)
name_set = df.index.unique(0) # a convenience

costs = df['Salary'].to_dict()
values = df['DK Ceiling'].to_dict()

# set up LP
draft = pulp.LpVariable.dicts('selected', legal_assignments, cat='Binary')

prob = pulp.LpProblem('the draft', LpMaximize)

#obj
prob += pulp.lpSum([draft[n,p]*values[n,p] for (n,p) in legal_assignments])

#salary cap
prob += pulp.lpSum([draft[n,p]*costs[n,p] for (n, p) in legal_assignments]) <= 38500

# positions
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PG']) <= 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SG']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SF']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PF']) == 0
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'C']) == 0
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'G']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'F']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'UTIL']) == 1

# use each player at most only once
result = []
for name in name_set:
    prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if n == name]) <= 1

prob.solve()

for idx in draft:
    if draft[idx].varValue > 0:
        print(idx[0], idx[1])
        
        result.append([idx[0], idx[1]])

Jordan Nwora (21919375) SF
Brandon Clarke (21918842) F
Lindy Waters III (21918789) G
De'Anthony Melton (21918612) PG
Nikola Jokic (21918253) UTIL
Cade Cunningham (21918310) SG


In [3]:
# Change dataframe from column to row

new_list = result
df = pd.DataFrame(new_list)
test = df[[1,0]]
test1 = test.transpose()
test2 = test1.rename(columns=test1.iloc[0])
test3 = test2.drop([1])
df2 = test3[['PG', 'SG','SF', 'PF', 'C', 'G', 'F', 'UTIL']]
df2.to_csv("dkng_result_" + time + str(today)  + ".csv", index=None)
# os.system("dkng_result_" + time + str(today)  + ".csv")

NameError: name 'result' is not defined

In [26]:
df2 = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\dkng_result_' + time + str(today)  + ".csv")

upload = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\DraftKings\Template\DK_Upload_PlayerPool_' + time + str(today)  + ".csv"
                      , #usecols=[0,1,2,3], 
                     error_bad_lines=False, engine ='python')

upload = upload[['Entry ID', 'Contest Name', 'Contest ID', 'Entry Fee']]
upload = upload.dropna()
# Drops rows that have NaN
upload2 = upload.dropna()
# creates value for row counts in the upload file
repeat_rows = len(upload2.index)

# duplicates lineups down rows based on number of contests (length of rows)
# copies columns from old data frame onto new dataframe
df3 = pd.DataFrame(np.repeat(df2.values, repeat_rows, axis=0))
df3.columns = df2.columns

# joins the two data frames
os.chdir(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\DraftKings\FinalUpload')
result = pd.concat([upload2, df3], axis=1)

# saves the dataframe
result.to_csv("DK_Final_Upload_" + time + str(today) +".csv", index=False)
os.system("DK_Final_Upload_"+ time + str(today) +".csv")

Skipping line 8: Expected 14 fields in line 8, saw 22
Skipping line 9: Expected 14 fields in line 9, saw 22
Skipping line 10: Expected 14 fields in line 10, saw 22
Skipping line 11: Expected 14 fields in line 11, saw 22
Skipping line 12: Expected 14 fields in line 12, saw 22
Skipping line 13: Expected 14 fields in line 13, saw 22
Skipping line 14: Expected 14 fields in line 14, saw 22
Skipping line 15: Expected 14 fields in line 15, saw 22
Skipping line 16: Expected 14 fields in line 16, saw 22
Skipping line 17: Expected 14 fields in line 17, saw 22
Skipping line 18: Expected 14 fields in line 18, saw 22
Skipping line 19: Expected 14 fields in line 19, saw 22
Skipping line 20: Expected 14 fields in line 20, saw 22
Skipping line 21: Expected 14 fields in line 21, saw 22
Skipping line 22: Expected 14 fields in line 22, saw 22
Skipping line 23: Expected 14 fields in line 23, saw 22
Skipping line 24: Expected 14 fields in line 24, saw 22
Skipping line 25: Expected 14 fields in line 25, saw

0

In [ ]:
upload